<a href="https://colab.research.google.com/github/thiagolermen/machine-learning-portifolio/blob/main/src/6-k-nearest-neighbor/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## K-Nearest Neighbor - KNN